In [11]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats as ss
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts
import basefunction.vBaseFunctions3
from tqdm import tqdm
import itertools

In [12]:
def save_fig(folder_path,name):
    import os
    old_path = os.getcwd()
    os.chdir(folder_path)
    plt.savefig(name)
    os.chdir(old_path)

def load_ripples():
    ripples_ = scipy.io.loadmat(f'{bk.load.session}-RippleFiring.mat')['ripples']['allsws'][0][0]
#     ripples_ = pd.DataFrame(data = ripples,columns=['start','peak','stop'])
    
    columns = ['start','peak','stop']
    
    ripples = {}
    for i,c in zip(range(ripples_.shape[1]),columns):
        ripples.update({c:nts.Ts(ripples_[:,i],time_units='s')})
    return ripples

def poissonTest(baseRate,counts,time):
    eps = np.spacing(1)
    
    lam = baseRate*time
    
    pInc = 1 - ss.poisson.cdf(counts,lam)
    pDec = ss.poisson.cdf(counts,lam)
    surprise = np.log((pDec+eps)/(pInc + eps))
    return pInc,pDec,surprise

def psth(neurons,stimulus,binSize,win):
    if isinstance(neurons,nts.time_series.Tsd): 
        neurons = np.array(neurons,'object')
    winLen = int((win[1] - win[0])/binSize)
    window = np.arange(winLen,dtype = int)-int(winLen/2)
    stim_bin = (stimulus/binSize).astype('int')
    t,binned = bk.compute.binSpikes(neurons,binSize,start = 0, stop = stimulus[-1]+win[-1])
    psth = np.empty((stimulus.size,len(neurons),winLen))
    
    for i,t in tqdm(enumerate(stim_bin)):
        psth[i] = binned[:,t+window]
        
    psth = np.mean(psth,0).T
    t = window*binSize
    return t,psth

# def correlogram(neuron1,neuron2,binSize,win):
    

In [24]:
def ripple_psth(path):  
    
    win = [-0.5,0.5]
    binSize = 0.05
    
    
    bk.load.current_session(path)
    ripples = load_ripples()
    
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)
    
    ripples_ = ripples['peak'].as_units('s').index.values
    t,psth_ = psth(neurons,ripples_,binSize,win)
    
    states = bk.load.states()
    states.update({'ripples':nts.IntervalSet(ripples['start'].times(),ripples['stop'].times())})
    states.update({'outside_ripples':states['sws'].set_diff(nts.IntervalSet(states['ripples'].start-200_000,states['ripples'].end+200_000,force_no_fix=True)).drop_short_intervals(1)})
    
    bk.plot.intervals(states['outside_ripples'],'orange')
    bk.plot.intervals(states['ripples'],'red')
    
    
    pInc = np.empty(len(neurons))
    pDec = np.empty(len(neurons))
    surprise = np.empty(len(neurons))
    for i,n in enumerate(neurons):
        baseRate = len(n.restrict(states['outside_ripples']))/states['outside_ripples'].tot_length(time_units = 's')
        count = len(n.restrict(states['ripples']))
        time = states['ripples'].tot_length(time_units = 's')
        
        pInc[i],pDec[i],surprise[i] = poissonTest(baseRate,count,time)
    
    stats = pd.DataFrame(np.array([pInc,pDec,surprise]).T,columns=['pInc','pDec','surprise'])
    
    metadata = pd.concat([metadata,stats],1) 
    
    print(metadata)
    return t,psth_,metadata

In [4]:
def ripple_psth_slow(path):
    nbins = 100
    
    
    bk.load.current_session(path)
    ripples = load_ripples()
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)

    win = [-0.5,0.5]
    n_neurons = len(neurons)
    
    peth = np.empty((len(ripples['peak'].as_units('s').index),nbins,n_neurons))
    
    for i,ripple_time in tqdm(enumerate(ripples['peak'].as_units('s').index)):
        t,binned = bk.compute.binSpikes(neurons,nbins=nbins,start = ripple_time+win[0],stop = ripple_time+win[1])
        peth[i,:] = binned.T
    
    
    states = bk.load.states()
    states.update({'ripples':nts.IntervalSet(ripples['start'].times(),ripples['stop'].times())})
    states.update({'outside_ripples':states['sws'].set_diff(nts.IntervalSet(states['ripples'].start-200_000,states['ripples'].end+200_000,force_no_fix=True))})
    
    pInc = np.empty(len(neurons))
    pDec = np.empty(len(neurons))
    surprise = np.empty(len(neurons))
    for i,n in enumerate(neurons):
        baseRate = len(n.restrict(states['outside_ripples']))/states['outside_ripples'].tot_length(time_units = 's')
        count = len(n.restrict(states['ripples']))
        time = states['ripples'].tot_length(time_units = 's')
        
        pInc[i],pDec[i],surprise[i] = poissonTest(baseRate,count,time)
    
    stats = pd.DataFrame(np.array([pInc,pDec,surprise]).T,columns=['pInc','pDec','surprise'])
    
    
    return t,peth,metadata,stats

In [22]:
%matplotlib qt

In [26]:
t,peth,meta = ripple_psth('Z:\Rat08\Rat08-20130718')

Rat : 8 on day : 10
Working with session Rat08-20130718 @ Z:\Rat08\Rat08-20130718


FileNotFoundError: [Errno 2] No such file or directory: 'Rat08-20130718-RippleFiring.mat'

In [19]:
batch = bk.load.batch(ripple_psth)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [04:17<00:00,  4.22s/it]
- |#                                                  | 0 Elapsed Time: 0:00:00


Batch finished in 257.462366104126
Some session were not processed correctly
['Rat08-20130718', 'Rat08-20130719', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140326', 'Rat09-20140327', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140403', 'Rat09-20140404', 'Rat09-20140405', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat11-20150325', 'Rat11-20150326', 'Rat11-20150327', 'Rat11-20150328', 'Rat11-20150330', 'Rat11-20150331', 'Rat11-20150401', 'Rat11-20150402', 'Rat11-20150403']
45.90163934426229 %


In [ ]:
metaall = pd.concat([batch[s][2] for s in batch])
psthall = np.hstack([batch[s][1] for s in batch])

In [40]:
meta

,Rat,Day,Shank,Id,Region,Type,pInc,pDec,surprise
0,8,6,1,2,Hpc,Pyr,0.000000e+00,1.000000,36.043653
1,8,6,2,2,Hpc,Pyr,1.670886e-13,1.000000,29.418924
2,8,6,2,3,Hpc,Pyr,0.000000e+00,1.000000,36.043653
3,8,6,2,4,Hpc,Pyr,0.000000e+00,1.000000,36.043653
4,8,6,2,5,Hpc,Pyr,0.000000e+00,1.000000,36.043653
...,...,...,...,...,...,...,...,...,...
247,8,6,19,2,BLA,Pyr,9.993548e-01,0.000645,-7.345331
248,8,6,19,3,BLA,Pyr,8.843166e-01,0.115683,-2.033958
249,8,6,19,4,BLA,Pyr,1.951515e-01,0.804848,1.416878
250,8,6,19,5,BLA,Pyr,2.487088e-01,0.751291,1.105511


In [55]:
stru = ['BLA','Hpc']
types = ['Pyr','Int']

combi = itertools.product(stru,types)
# print(list(combi))

for s,t in combi:
    mask = (meta.Region == s) & (meta.Type == t)
    print(len(meta[mask]))
    plt.figure()
    plt.title(s+' '+t)
    plt.plot(peth[:,mask])

83
11
55
5


In [5]:
%time t,p = psth(neurons,neurons[0].as_units('s').index.values,0.001,[-0.1,0.1])

NameError: name 'neurons' is not defined

In [6]:
bk.load.current_session('Z:\Rat08\Rat08-20130708')

Rat : 8 on day : 1
Working with session Rat08-20130708 @ Z:\Rat08\Rat08-20130708


True

In [64]:
p = batch['Rat08-20130708'][1]

In [65]:
p

array([[0.04365711, 0.03834926, 0.02229299, ..., 0.25968684, 0.17794586,
        0.06595011],
       [0.04126858, 0.03914544, 0.02162951, ..., 0.25849257, 0.17781316,
        0.06011146],
       [0.04206476, 0.03821656, 0.02162951, ..., 0.24787686, 0.17091295,
        0.05931529],
       ...,
       [0.02773355, 0.04617834, 0.02308917, ..., 0.26539278, 0.18285563,
        0.05533439],
       [0.03038747, 0.04020701, 0.02534501, ..., 0.26366773, 0.18285563,
        0.05573248],
       [0.03516454, 0.04299363, 0.02149682, ..., 0.26141189, 0.18444798,
        0.06581741]])

In [7]:
test = ripple_psth(bk.load.path)

Rat : 8 on day : 1
Working with session Rat08-20130708 @ Z:\Rat08\Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy


7536it [00:00, 12162.80it/s]


    Rat Day Shank Id Region Type          pInc          pDec   surprise
0     8   1     4  2    Hpc  Pyr  1.000000e+00  6.904731e-27 -36.043653
1     8   1     4  3    Hpc  Pyr  0.000000e+00  1.000000e+00  36.043653
2     8   1     4  4    Hpc  Int  0.000000e+00  1.000000e+00  36.043653
3     8   1     4  5    Hpc  Pyr  0.000000e+00  1.000000e+00  36.043653
4     8   1     4  6    Hpc  Pyr  0.000000e+00  1.000000e+00  36.043653
..   ..  ..   ... ..    ...  ...           ...           ...        ...
161   8   1    20  2   None  Pyr  3.168839e-01  6.831161e-01   0.768129
162   8   1    20  3   None  Unk  9.820839e-01  1.791610e-02  -4.003977
163   8   1    20  4   None  Unk  0.000000e+00  1.000000e+00  36.043653
164   8   1    20  5   None  Unk  1.022152e-07  9.999999e-01  16.096186
165   8   1    20  6   None  Int  1.360023e-13  1.000000e+00  29.624473

[166 rows x 9 columns]


In [ ]:
batch2 = bk.load.batch(ripple_psth)

  3%|██████▌                                                                                                                                                                                                 | 2/61 [00:53<24:21, 24.77s/it]

Loading Data from Rat08-20130710
Rat : 8 on day : 3
Working with session Rat08-20130710 @ Z:\Rat08\Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


In [ ]:
import seaborn as sns

In [ ]:
%matplotlib qt

In [ ]:
plt.title('Surprise Neurons')

In [ ]:
sns.violinplot(metaall[metaall['Region'] != 'None']['Region'],metaall[metaall['Region'] != 'None']['surprise'])

In [ ]:
batch['Rat08-20130708'][2]

In [ ]:
temps = batch2['Rat08-20130710'][0]

In [ ]:
%matplotlib qt

In [ ]:
np.

In [ ]:
scipy.version.full_version

In [ ]:
stru = np.unique(metaall.Region)
types = ['Pyr']
stru = ['Pir']
for s,t in itertools.product(stru,types):
    mask = (metaall.pInc < 0.0001) & (metaall.Region == s) & (metaall.Type == t)
    mask = list(mask)
    psth = psthall[:,np.where(mask)[0]]
    
#     plt.imshow(scipy.stats.zscore(psth).T)
    for i,p in enumerate(psth.T):
        if i > 25: break
        zscore_p = scipy.stats.zscore(p)
        g = scipy.signal.gaussian(10,1)
        g = g/np.sum(g)
        smooth_p = np.convolve(zscore_p,g,'same')
        
        plt.figure()
        plt.bar(temps,zscore_p,temps[1]-temps[0])
        plt.plot(temps,smooth_p,'r--')

In [ ]:
stru = np.unique(metaall.Region)
types = ['Pyr','Int']
stru = ['Hpc','BLA']
for s,t in itertools.product(stru,types):
    mask = (metaall.pInc < 0.001) & (metaall.Region == s) & (metaall.Type == t)
    mask = list(mask)
    
    psth_stru_types_u = np.nanmean(psthall[:,np.where(mask)[0]],1)
    
    mask = (metaall.pDec < 0.001) & (metaall.Region == s) & (metaall.Type == t)
    mask = list(mask)
    psth_stru_types_d = np.nanmean(psthall[:,np.where(mask)[0]],1)
    
    plt.figure()
    
    plt.suptitle(f'{s} and {t}')
    
    plt.subplot(1,2,1)
    plt.title('Up-Mod')
    plt.bar(temps,psth_stru_types_u,0.002,alpha = 0.5)
    plt.axvline(0,color = 'r',alpha = 0.5)
    
    plt.xlim(-0.5,0.5)
    
    plt.subplot(1,2,2)
    plt.title('Down-Mod')
    plt.bar(temps,psth_stru_types_d,0.002,alpha = 0.5)
    plt.axvline(0,color = 'r',alpha = 0.5)
    plt.xlim(-0.5,0.5)

    
    plt.tight_layout()
#     save_fig('P:/Data/Figures/',f'{s}-{t}-0.002-full.svg')
#     save_fig('P:/Data/Figures/',f'{s}-{t}-0.002-full.png')
    